# Get started

LCEL이란 ?
LangChain Expression Language의 줄임말로, 프로토타입에서 생산으로의 전환을 쉽게 지원합니다. 스트리밍 및 비동기 지원, 병렬 실행 최적화, 재시도 및 대체 방법, 중간 결과 접근, 입력/출력 스키마 등을 특징으로 합니다.

#### 필수 라이브러리 설치

In [44]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

#### OpenAI API Key 등록

In [53]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-****************************************************"

## Basic example: prompt + model + output parser

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

### 1. Prompt

In [57]:
#Prompt Message 확인하기
prompt_value = prompt.invoke({"topic": "ice cream"})

In [58]:
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [59]:
#Prompt Message 추출
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [60]:
#Prompt String 으로 content만 추출
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

### 2. Model

In [ ]:
#Model에 Prompt 넣어서 실행
message = model.invoke(prompt_value)

In [62]:
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [63]:
from langchain_openai.llms import OpenAI

#LLM Model Test
llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nWhy did the ice cream go to therapy? Because it was feeling a little melon-coly.'

### 3. Output parser

In [64]:
#출력값 확인
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

### 4. Entire Pipeline

In [65]:
input = {"topic": "ice cream"}

In [66]:
#Chain 실행
(prompt | model).invoke(input)

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're on the dance floor!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

## RAG Search Example

#### 필수 라이브러리 다운로드

In [67]:
!pip --q install langchain docarray tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 4.4 MB/s eta 0:00:00


In [68]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

#Text로 VectorRetriever 생성
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

#Prompt 템플릿 생성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

#모델 설정
model = ChatOpenAI()

#String 데이터 타입의 Output Parser 생성
output_parser = StrOutputParser()

#Retrieval Runnable 생성
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

#LCEL Chain 생성
chain = setup_and_retrieval | prompt | model | output_parser


/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [69]:
#Chain 실행 및 답변 확인
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'